In [1]:
import apache_beam as beam  
import os
import shutil
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.coders import example_proto_coder
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.tf_metadata import dataset_schema

In [2]:
import tensorflow as tf

Edit the following, **replacing `<YOUR_BUCKET_PATH>` and `<WORKFLOW_NAME>` with the correct values**.  You can find the name of a given workflow by looking at the `argo submit` output, or in the Argo UI.  The workflow name is also reflected in the Kubernetes pod names.

In [3]:
OUTPUT_PATH_PREFIX = 'gs://<YOUR_BUCKET_PATH>/<WORKFLOW_NAME>/'

In [4]:
tfma_result1 = tfma.load_eval_result(
    output_path= OUTPUT_PATH_PREFIX + 'tfma/output')
tfma_result2 = tfma.load_eval_result(
    output_path=OUTPUT_PATH_PREFIX + 'tfma2/output')

InvalidArgumentError: Error executing an HTTP request: HTTP response code 400 with body '{
  "error": {
    "code": 400,
    "message": "Invalid bucket name: '\u003cYOUR_BUCKET_PATH\u003e'",
    "errors": [
      {
        "message": "Invalid bucket name: '\u003cYOUR_BUCKET_PATH\u003e'",
        "domain": "global",
        "reason": "invalid"
      }
    ]
  }
}
'
	 when reading metadata of gs://<YOUR_BUCKET_PATH>/<WORKFLOW_NAME>/tfma/output/eval_config.json

In [ ]:
tfma.view.render_slicing_metrics(
    tfma_result1, slicing_column='trip_start_hour')

In [ ]:
tfma.view.render_slicing_metrics(
    tfma_result2, slicing_column='trip_start_hour')

In [ ]:

# An empty slice spec means the overall slice, that is, the whole dataset.# An em 
OVERALL_SLICE_SPEC = tfma.SingleSliceSpec()

# Data can be sliced along a feature column
# In this case, data is sliced along feature column trip_start_hour.
FEATURE_COLUMN_SLICE_SPEC = tfma.SingleSliceSpec(columns=['trip_start_hour'])

# Data can be sliced by crossing feature columns
# In this case, slices are computed for trip_start_day x trip_start_month.
FEATURE_COLUMN_CROSS_SPEC = tfma.SingleSliceSpec(columns=['trip_start_day', 'trip_start_month'])

# Metrics can be computed for a particular feature value.
# In this case, metrics is computed for all data where trip_start_hour is 12.
FEATURE_VALUE_SPEC = tfma.SingleSliceSpec(features=[('trip_start_hour', 12)])

# It is also possible to mix column cross and feature value cross.
# In this case, data where trip_start_hour is 12 will be sliced by trip_start_day.
COLUMN_CROSS_VALUE_SPEC = tfma.SingleSliceSpec(columns=['trip_start_day'], features=[('trip_start_hour', 12)])

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
    FEATURE_COLUMN_SLICE_SPEC, 
    FEATURE_COLUMN_CROSS_SPEC, 
    FEATURE_VALUE_SPEC, 
    COLUMN_CROSS_VALUE_SPEC    
]

In [ ]:
# Show metrics sliced by COLUMN_CROSS_VALUE_SPEC above.
tfma.view.render_slicing_metrics(tfma_result1, slicing_spec=COLUMN_CROSS_VALUE_SPEC)

In [ ]:
# Show overall metrics.
tfma.view.render_slicing_metrics(tfma_result1)

In [ ]:
# Show overall metrics.
tfma.view.render_slicing_metrics(tfma_result2)

In [ ]:
# Visualize the results in a Time Series. In this case, we are showing the slice specified.
eval_results_from_disk = tfma.load_eval_results([
    OUTPUT_PATH_PREFIX + 'tfma/output', 
    OUTPUT_PATH_PREFIX + 'tfma2/output'
], 
                                                tfma.constants.MODEL_CENTRIC_MODE)
tfma.view.render_time_series(eval_results_from_disk, FEATURE_VALUE_SPEC)

Copyright 2018 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.